### Check an endpoint
This notebook aims to run the pipeline on a given endpoint to check to see if it will be successful. This includes collecting, pipeline and datset stages. It aims to also highlight useful information as a summary as to whether the endpoint would be successful on our platform. it will download all relevant data to do this and hence might be disk intensive. You'll need to provide the following information:
- collection - this is the collection that the dataset belongs to, this can be extracted from the specification but for this notebook we ask to provide it incase you want to test the pipeline on something which isn't being included in the main site right now
- dataset - this is the dataset that the endpoint is meant to provide data for, technically this can be multiple datasets but this this use case it should just be one. It is also the name of the pipeline that is ran on the individual resources that are downloaded from the endpoint. the terms dataset/pipline are often the same
- organisation - the organisation identifier to be used for the endpoint
- endpoint url - the actual url needed for the endpoint
- plugin - often we use plugins to download the data this is only needed for specific endpoints

If you are seeing errors regarding digital-land, then try 

`pip install -e git+https://github.com/digital-land/pipeline.git#egg=digital-land`

And restart the notebook.

In [ ]:
import os
import pandas as pd
from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource
from digital_land.collection import Collection

In [ ]:
#southwark convservation area endpoint
# collection_name = 'conservation-area-collection'
# dataset = 'conservation-area'
# organisation = 'local-authority-eng:SWK'
# endpoint_url = 'https://www.southwark.gov.uk/assets/attach/194104/Conservation-Areas.gpkg'
# plugin = None
# additional_column_mappings=None
# additional_concats=None

# doncaster tpos
# collection_name = 'tree-preservation-order-collection'
# dataset = 'tree-preservation-zone'
# organisation = 'local-authority-eng:DNC'
# endpoint_url='https://maps.doncaster.gov.uk/server/rest/services/Planning/TPO_Map/MapServer/1'
# plugin = 'arcgis'
# additional_column_mappings=None
# additional_concats=None

# 
collection_name = 'listed-building-collection'
dataset = 'listed-building-outline'
organisation = 'local-authority-eng:CAM'
endpoint_url = 'https://opendata.camden.gov.uk/api/views/uu3n-zgbj/rows.csv?accessType=DOWNLOAD'
plugin = None

additional_column_mappings=None
additional_concats=None

# generic data_dir setting
data_dir = '../data/endpoint_checker'

# example playing with additional confiigs
# additional_concats = [{
#     'dataset':'tree-preservation-zone',
#     'endpoint':'de1eb90a8b037292ef8ae14bfabd1184847ef99b7c6296bb6e75379e6c1f9572',
#     'resource':'e6b0ccaf9b50a7f57a543484fd291dbe43f52a6231b681c3a7cc5e35a6aba254',
#     'field':'reference',
#     'fields':'REFVAL;LABEL',
#     'separator':'/'
# }]


In [ ]:
run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir,
    additional_col_mappings=additional_column_mappings,
    additional_concats=additional_concats
)

#### Collection log summaries

We need to establish if a resource was downloaded from the endpoint and whether there were any issues during the collection process. Examine the output of the below. There should be one log for the attempt made at downloading from the endpoint. If status code is 200 then the resource was downloaded successfully

In [ ]:
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))

collection.resource.records

In [ ]:

logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
logs

### Check unnassigned entiities
This process automatically aims to detect and assign entity numbers where entries are currently unnassigned. Examine the list below to see what (if any) entities have been assigned. if you were to include these in an actual pipeline you would need to update the configuration lookup.csv with these values. It's worth checking they are sensible before this happens 

In [ ]:
unassigned_entries = pd.read_csv(os.path.join(data_dir,'var','cache','unassigned-entries.csv'))
if len(unassigned_entries) == 0:
    print('no additional entity numbers where required')
else:
    print(unassigned_entries)

#### Check logs collated from the pipeline process
We need to read the logs and examine to see if the data points were all read in correctly. This uses the sqlite database to do so with some custom queries. You could directly examine the csvs if the pipeline fails.

First, check the column mappings to see what columns the pipeline automatically mapped. Tf this is empty or missing values, then it's likely to be the reason data isn't appearing at the end.

In [ ]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_column_mappings()

results

examine the issues logs, we'll look at the types of errors being raised and list all of them. This could be improved in the future by examining the severity

In [ ]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues_by_type()
results

In [ ]:
results = dataset_db.get_issues()


results.loc[(results.issue_type != 'default-value') ]

# duff_geom = results.loc[(results.issue_type == 'invalid geometry')]

duff_geom = results.loc[(results.issue_type == 'Unexpected geom type')]

cols = duff_geom[["line_number", "value"]]

print (cols)


#### Final dataset comparison against the sqlite database

Below are two tables which show the difference betwen what was provided to us and what is ucrrently in the entity table. It is important to bear in mind that we assign entities automaticallyis process, the table above shows what we have added.

In [ ]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_entities()
results.head(26)

In [ ]:
# load in raw resources
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
resources = collection.resource.entries
resources

In [ ]:
# currently this just reads in the raw resource but in  the future this should check for 
# converted resource first
resource = resources[0]['resource']
resource_path = os.path.join(data_dir,'collection','resource',resource)
try:
    raw_resource = pd.read_csv(resource_path)
except (UnicodeDecodeError,TypeError,pd.errors.ParserError):
    converted_resource_dir = os.path.join(data_dir,'var','converted_resources')
    converted_resource_path = os.path.join(converted_resource_dir,f'{resource}.csv') 
    if not os.path.exists(converted_resource_path):
        convert_resource(resource,resource_path,converted_resource_dir,dataset)
    raw_resource = pd.read_csv(converted_resource_path)
    

In [ ]:
raw_resource